In [77]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image

num_epochs = 20
batch_size = 64

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
trainset = tv.datasets.MNIST(root='./data',  train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        self.encoder = nn.Sequential(
            # 28 x 28
            nn.Conv2d(1, 4, kernel_size=5),
#             nn.Dropout2d(p=0.2),
            # 4 x 24 x 24
            nn.ReLU(True),
            nn.Conv2d(4, 8, kernel_size=5),
            nn.ReLU(True),
#             nn.Dropout2d(p=0.2),
            # 8 x 20 x 20 = 3200
            nn.Flatten(),
            nn.Linear(3200, 10),
            # 10
            nn.Softmax(),
            )
        self.decoder = nn.Sequential(
            # 10
            nn.Linear(10, 400),
            # 400
            nn.ReLU(True),
            nn.Linear(400, 4000),
            # 4000
            nn.ReLU(True),
            nn.Unflatten(1, (10, 20, 20)),
            # 10 x 20 x 20
#             nn.Dropout2d(p=0.2),
            nn.ConvTranspose2d(10, 10, kernel_size=5),
            # 24 x 24
#             nn.ReLU(True),
#             nn.Dropout2d(p=0.2),
            nn.ConvTranspose2d(10, 1, kernel_size=5),
            # 28 x 28
            #nn.ReLU(True),
            nn.Sigmoid(),
            )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = Autoencoder().cpu()
distance = nn.MSELoss()
#distance = nn.BCELoss()
#distance = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-5)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img).cpu()
        output = model(img)
        loss = distance(output, img)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('epoch [{}/{}], loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

epoch [1/20], loss: 0.8514
epoch [2/20], loss: 0.8257
epoch [3/20], loss: 0.7773
epoch [4/20], loss: 0.7251
epoch [5/20], loss: 0.7215
epoch [6/20], loss: 0.7001
epoch [7/20], loss: 0.7056
epoch [8/20], loss: 0.7124
epoch [9/20], loss: 0.6659
epoch [10/20], loss: 0.6738
epoch [11/20], loss: 0.6583
epoch [12/20], loss: 0.7155
epoch [13/20], loss: 0.6189
epoch [14/20], loss: 0.6356
epoch [15/20], loss: 0.6755
epoch [16/20], loss: 0.6922
epoch [17/20], loss: 0.6262
epoch [18/20], loss: 0.6399
epoch [19/20], loss: 0.6440
epoch [20/20], loss: 0.6444


In [82]:
import numpy as np
np.set_printoptions(suppress=True)

confusion_matrix = np.zeros((10, 10))

batch_size = 20*1000

testset = tv.datasets.MNIST(root='./data',  train=False, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=4)

for data in dataloader:
    imgs, labels = data
    imgs = Variable(imgs).cpu()
    encs = model.encoder(imgs).detach().numpy()
    for i in range(len(encs)):
        predicted = np.argmax(encs[i])
        actual = labels[i]
        confusion_matrix[actual][predicted] += 1
print(confusion_matrix)
print(int(confusion_matrix.sum()))

[[642.   0. 147.  37. 105.   0.   0.   2.  47.   0.]
 [  0.   0. 496. 171.   1.   0.   0. 119.   5. 343.]
 [203.   0. 257.  36.   4.   0.   0. 146. 250. 136.]
 [ 91.   0.  47. 107.  19.   0.   0.  88. 397. 261.]
 [ 12.   0.  94.  45. 758.   0.   0.   5.   0.  68.]
 [ 54.   0. 194.  65. 252.   0.   0.   9. 204. 114.]
 [ 91.   0.  98. 734.   8.   0.   0.   5.   0.  22.]
 [  0.   0.  12.   6. 283.   0.   0. 659.  12.  56.]
 [ 44.   0.  78.  51. 178.   0.   0.   6. 512. 105.]
 [ 12.   0.  17.   8. 870.   0.   0.  13.  11.  78.]]
10000


In [91]:
for actual in range(10):
    predicted = np.argmax(confusion_matrix[actual])
    pct = 100 * confusion_matrix[actual][predicted] / np.sum(confusion_matrix[actual])
    print('Actual label %d most likely activates on neuron %d (%d%%)' % (actual, predicted, pct))

Actual label 0 most likely activates on neuron 0 (65%)
Actual label 1 most likely activates on neuron 2 (43%)
Actual label 2 most likely activates on neuron 2 (24%)
Actual label 3 most likely activates on neuron 8 (39%)
Actual label 4 most likely activates on neuron 4 (77%)
Actual label 5 most likely activates on neuron 4 (28%)
Actual label 6 most likely activates on neuron 3 (76%)
Actual label 7 most likely activates on neuron 7 (64%)
Actual label 8 most likely activates on neuron 8 (52%)
Actual label 9 most likely activates on neuron 4 (86%)
